# 텐서플로 2.0 시작하기: 전문가용

In [23]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [24]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [25]:
x_train.shape

(60000, 28, 28)

In [26]:
# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [27]:
x_train.shape

(60000, 28, 28, 1)

Tensorflow | tf.data.Dataset.from_tensor_slices()
- we can get the slices of an array in the form of objects by using

In [29]:
gfg = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [35]:
gfg.element_spec

(TensorSpec(shape=(28, 28, 1), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.uint8, name=None))

In [36]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [39]:
train_ds.element_spec

(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None),
 TensorSpec(shape=(None,), dtype=tf.uint8, name=None))

## Keras subclassing 

In [42]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

`Conv2D`
two parameters
- filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
- kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions.

In [43]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

## 최종 결과를 출력

In [44]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### tf.GradientTape를 사용하여 모델을 훈련

In [45]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) # 
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) #

    train_loss(loss)
    train_accuracy(labels, predictions)

### 모델을 테스트

In [46]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

### 결과

In [47]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
        
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))        

에포크: 1, 손실: 0.12867332060008, 정확도: 96.06666666666666, 테스트 손실: 0.06900014001195477, 테스트 정확도: 97.77
에포크: 2, 손실: 0.08521700846002593, 정확도: 97.405, 테스트 손실: 0.0638097174536533, 테스트 정확도: 97.95
에포크: 3, 손실: 0.06439093441802089, 정확도: 98.00999999999999, 테스트 손실: 0.06106548545560427, 테스트 정확도: 98.03
에포크: 4, 손실: 0.051436655622114025, 정확도: 98.39041666666667, 테스트 손실: 0.06045021839570492, 테스트 정확도: 98.095
에포크: 5, 손실: 0.04311001509322057, 정확도: 98.64133333333334, 테스트 손실: 0.06057658664501114, 테스트 정확도: 98.146
